# Langchain crash course

In [ ]:
!pip install openai
!pip install langchain
!pip install google-search-results
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 6.4 MB/s e

In [ ]:
# from secret_key import openapi_key
# import os
# os.environ['OPENAI_API_KEY'] = openapi_key

import os
os.environ['OPENAI_API_KEY'] = "your api key"

## LLMs

In [ ]:
!pip install langchain

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.invoke("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)


"Saffron Spice Palace"


In [ ]:
llm("I want to open a restaurant for Kashmiri food. Suggest a fency name for this.")

'\n\n"Kashmiri Spice Bazaar"'

## Prompt Templates

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Pakistani")
print(p)

print(llm.predict(p))

I want to open a restaurant for Pakistani food. Suggest a fency name for this.


"Spice Kingdom: A Taste of Pakistan"


## Chains

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.invoke("Arabic")

{'cuisine': 'Arabic', 'text': '\n\n"Al-Fakhara" (The Elegance)'}

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.invoke("Arabic")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Arabic food. Suggest a fency name for this.

> Finished chain.


{'cuisine': 'Arabic',
 'text': '\n\n"Al-Khayma" (which means "The Tent" in Arabic)'}

In [ ]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [ ]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Kashmiri")
print(content)



1. Rogan Josh (Kashmiri lamb curry)
2. Chicken Korma (creamy chicken curry)
3. Saag Paneer (spinach and cheese curry)
4. Kashmiri Pulao (spiced rice with dried fruits and nuts)
5. Tandoori Chicken (marinated and grilled chicken)
6. Kashmiri Naan (stuffed with dried fruits and nuts)
7. Dum Aloo (spicy stuffed potatoes)
8. Goshtaba (Kashmiri meatballs in yogurt gravy)
9. Aloo Palak (potatoes and spinach curry)
10. Paneer Tikka Masala (grilled cheese in tomato cream sauce)
11. Dal Makhani (creamy lentil curry)
12. Vegetable Biryani (spiced rice with mixed vegetables)
13. Kashmiri Chai (traditional tea with spices and saffron)
14. Mango Lassi (refreshing yogurt drink)
15. Gulab Jamun (syrup-soaked fried dough balls).


#### Sequential Chain

In [ ]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [ ]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [ ]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [ ]:
chain({"cuisine": "Kashmiri"})

{'cuisine': 'Kashmiri',
 'restaurant_name': '\n\n"Kashmiri Spice Bazaar"',
 'menu_items': '\n\n1. Kashmiri Chicken Kebabs\n2. Rogan Josh (Lamb Curry)\n3. Vegetable Samosas\n4. Kashmiri Pulao (Rice dish with nuts and dried fruits)\n5. Tandoori Naan\n6. Chicken Biryani\n7. Lamb Vindaloo\n8. Kashmiri Dum Aloo (Potatoes in spicy tomato-based sauce)\n9. Palak Paneer (Spinach and cottage cheese curry)\n10. Kashmiri Kofta (Spiced meatballs in rich gravy)\n11. Dal Makhani (Creamy lentil dish)\n12. Kashmiri Chicken Yakhni (Chicken in yogurt-based gravy)\n13. Aloo Gobi (Potato and cauliflower stir fry)\n14. Kashmiri Kesar Kheer (Rice pudding with saffron and dried fruits)\n15. Gulab Jamun (Fried dough balls in syrup)'}

## Agents

In [ ]:
# make sure yoy have installed this package: pip install google-search-results
# from secret_key import serpapi_key
# os.environ['SERPAPI_API_KEY'] = serpapi_key

os.environ['SERPAPI_API_KEY'] = "a638f58a8e99ee10490da3fa15032843a718119a1c46a3099f047613706c252e"

#### Serpapi and llm-math tool

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.invoke("What was the GDP of Pakistan in 2022?")



> Entering new AgentExecutor chain...
 I should search for the GDP of Pakistan in 2022
Action: Search
Action Input: "GDP of Pakistan 2022"
Observation: $353 Billion US dollars
Thought: I should use a calculator to convert the GDP to another currency if needed
Action: Calculator
Action Input: $353 Billion
Observation: Answer: 353000000000
Thought: I now know the final answer
Final Answer: The GDP of Pakistan in 2022 is $353 Billion US dollars.

> Finished chain.


{'input': 'What was the GDP of Pakistan in 2022?',
 'output': 'The GDP of Pakistan in 2022 is $353 Billion US dollars.'}

#### Wikipedia and llm-math tool

In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=18e4c8f10f49e7e040b8e519aa53f6a6e76c6209c49ad85c5a96a0299ca31c2c
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk's birthdate and current age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is the wealthiest person in the world, with an estimated net worth of US$232 billion as of December 2023, according to the Bloomberg Billionaires Index, and $254 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship 

'Elon Musk will be 52 years old in 2023.'

## Memory

In [ ]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Kashmiri")
print(name)



"Kashmiri Spice Bazaar"


In [ ]:
name = chain.run("Pakistani")
print(name)



"Spice of Pakistan"


In [ ]:
chain.memory

In [ ]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



"El Sabor Exquisito" (The Exquisite Flavor)


In [ ]:
name = chain.run("Arabic")
print(name)



"Al-Fawarish" (meaning "The Treasures" in Arabic)


In [ ]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor Exquisito" (The Exquisite Flavor)
Human: Arabic
AI: 

"Al-Fawarish" (meaning "The Treasures" in Arabic)


#### ConversationChain

In [ ]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [ ]:
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was held in 1975 and was won by the West Indies team. The final match was played between the West Indies and Australia, with the West Indies team winning by 17 runs. It was a historic moment for the West Indies as it was their first ever world cup win. The player of the tournament was Clive Lloyd, the captain of the West Indies team.'

In [ ]:
convo.run("How much is 5+5?")

'  The answer to 5+5 is 10. It is a basic addition problem and the result is a whole number. The concept of addition is one of the fundamental operations in mathematics and is used in many everyday situations, from counting objects to calculating sums of money. In the decimal number system, 10 is represented as two digits, 1 and 0.'

In [ ]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the winning team in the first cricket world cup was Clive Lloyd. He was a left-handed batsman and a part of the West Indies cricket team from 1966 to 1985. He was known for his aggressive and powerful batting style and was also a strong leader, leading the West Indies team to many victories. In the first cricket world cup, he scored 102 runs in the final match and was named the player of the tournament.'

In [ ]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was held in 1975 and was won by the West Indies team. The final match was played between the West Indies and Australia, with the West Indies team winning by 17 runs. It was a historic moment for the West Indies as it was their first ever world cup win. The player of the tournament was Clive Lloyd, the captain of the West Indies team.
Human: How much is 5+5?
AI:   The answer to 5+5 is 10. It is a basic addition problem and the result is a whole number. The concept of addition is one of the fundamental operations in mathematics and is used in many everyday situations, from counting objects to calculating sums of money. In the decimal number system, 10 is represented as two digits, 1 and 0.
Human: Who was the captain ofthe winning team?
AI:  The captain of the winning team in the first cricket world cup was Clive Lloyd. He was a left-handed batsman and a part of the West Indies cricket team from 1966 to 1985. He 

#### ConversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was won by the West Indies in 1975. They defeated Australia in the final match by 17 runs. The tournament was held in England and was organized by the International Cricket Council (ICC). The West Indies team, led by captain Clive Lloyd, consisted of players such as Viv Richards, Gordon Greenidge, and Michael Holding. This was the inaugural cricket world cup and it consisted of eight teams from around the world. The West Indies were considered the favorites going into the tournament and they lived up to expectations by winning the championship.'

In [ ]:
convo.run("How much is 5+5?")

' The answer to 5+5 is 10.'

In [ ]:
convo.run("Who was the captain of the winning team?")

" I don't have enough context to accurately answer that question. Can you provide more information?"